In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [ ]:
!pip install -q \
  transformers>=4.38.0 \
  peft>=0.10.0 \
  accelerate>=0.26.0 \
  bitsandbytes>=0.43.0 \
  datasets \
  sentencepiece


In [ ]:
from huggingface_hub import login
login()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "Qwen/Qwen3-4B-Instruct-2507"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=False
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Apziva/ProjectC.csv")
df = df.drop_duplicates(subset=['job_title'])
df = df.dropna(subset=['job_title'])
job_titles = df.iloc[:, 1].tolist()
print(job_titles)
print(len(job_titles))

def run_llm(model, tokenizer, messages, max_tokens=100):

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_tokens
    )

    # skip input
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(
        generated_ids,
        skip_special_tokens=True
    )[0]

    return response

def build_score_prompt(title):
    role_keywords = ["Aspiring Human Resources", "seeking human resources"]
    prompt = f"""
    You are a professional career ranking expert.

    Your task is to evaluate candidates based on their job title.
    The role we are hiring for is: Aspiring human resources or seeking human resources.
    Only consider job titles that match this role or are closely related.

    Job titles that are unrelated to the role should get 1.
    Job titles that are slightly related but not ideal should get 2 or 3.
    Only titles that are fully relevant get 4 or 5.

    Examples:
    - "Aspiring Human Resources Professional" -> 5
    - "HR Generalist" -> 4
    - "Business Analyst" -> 2
    - "English Teacher" -> 1

    Evaluate the fit of this job title for the given role and give a score from 1 (low) to 5 (high).
    Only output the score.

    Job Title: {title}
    Role Keywords: {', '.join(role_keywords)}
    """

    messages = [
    {"role": "system", "content": "You are a professional career ranking expert, focusing on assessing candidate fit for a specified role."},
    {"role": "user", "content": prompt}
    ]
    return messages


scores = []
for title in job_titles:
    messages = build_score_prompt(title)
    response = run_llm(model, tokenizer, messages)
    try:
        score = int(response.strip())
    except:
        score = None
    scores.append(score)

df["rough_score"] = scores

print(df)

Mounted at /content/drive
['2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional', 'Native English Teacher at EPIK (English Program in Korea)', 'Aspiring Human Resources Professional', 'People Development Coordinator at Ryan', 'Advisory Board Member at Celal Bayar University', 'Aspiring Human Resources Specialist', 'Student at Humber College and Aspiring Human Resources Generalist', 'HR Senior Specialist', 'Seeking Human Resources HRIS and Generalist Positions', 'Student at Chapman University', 'SVP, CHRO, Marketing & Communications, CSR Officer | ENGIE | Houston | The Woodlands | Energy | GPHR | SPHR', 'Human Resources Coordinator at InterContinental Buckhead Atlanta', 'Aspiring Human Resources Management student seeking an internship', 'Seeking Human Resources Opportunities', 'Experienced Retail Manager and aspiring Human Resources Professional', 'Human Resources, Staffing and Recruiting Professional', 'Human Resources Specialist at

In [ ]:
df_unique = df.drop_duplicates(subset="job_title").copy()
total_train = 50
score_counts = df_unique['rough_score'].value_counts().sort_index()
print("Counts per score:", score_counts.to_dict())
train_samples = []
for score, count in score_counts.items():
    n_samples = round(total_train * count / len(df_unique))
    n_samples = min(n_samples, count)

    if n_samples > 0:
        sampled = df_unique[df_unique['rough_score'] == score].sample(n=n_samples, random_state=42)
        train_samples.append(sampled)

df_train = pd.concat(train_samples).reset_index(drop=True)
print(f"Training candidates selected: {len(df_train)}")
df_train[['job_title','rough_score']]
print(df_train)
print(len(df_train))

Counts per score: {1: 15, 2: 5, 3: 1, 4: 12, 5: 19}
Training candidates selected: 50
     id                                          job_title  \
0    93  Admissions Representative at Community medical...   
1    98                                            Student   
2     2  Native English Teacher at EPIK (English Progra...   
3   103                     Always set them up for Success   
4    87  Bachelor of Science in Biology from Victoria U...   
5    92  Seeking employment opportunities within Custom...   
6    11                      Student at Chapman University   
7     5    Advisory Board Member at Celal Bayar University   
8   104   Director Of Administration at Excellence Logging   
9    80           Junior MES Engineer| Information Systems   
10   91       Lead Official at Western Illinois University   
11   95              Student at Westfield State University   
12  102   Business Intelligence and Analytics at Travelers   
13   12  SVP, CHRO, Marketing & Communications,

In [ ]:
def build_pair_prompt(title_a, title_b, role="Aspiring Human Resources", role_keywords=None):
    """
    construct LLM messages，compare job_titles in pair。
    role: the role we are hiring for
    title_a: job title A
    title_b: job title B
    role_keywords: list[str] keywords that describe the role
    """

    if role_keywords is None:
        role_keywords = ["Aspiring Human Resources", "seeking human resources"]

    prompt = (f'''
        You are a professional career ranking expert.\n"
        The role we are hiring for is: {role}.\n"
        Only consider job titles that match this role or are closely related. \n"
        Ignore titles unrelated to HR, even if they contain other skills or experiences.\n"
        Compare these two job titles and determine which candidate is more fit for the role.\n"
        A: {title_a}\n"
        B: {title_b}\n"
        Role Keywords: {', '.join(role_keywords)}\n"
        Only answer 'A' if title A is more fit, or 'B' if title B is more fit."
        ''')

    messages = [
        {"role": "system", "content": "You are a professional career ranking assistant, focusing on assessing candidate fit for a specified role."},
        {"role": "user", "content": prompt}
    ]
    return messages

from itertools import combinations
import json

titles = df_train["job_title"].drop_duplicates().tolist()
print(f"Number of training titles: {len(titles)}")

pairwise_results = []

for title_a, title_b in combinations(titles, 2):

    # 先获取 rough_score
    score_a = df_train[df_train['job_title'] == title_a]['rough_score'].values[0]
    score_b = df_train[df_train['job_title'] == title_b]['rough_score'].values[0]

    # 先用 rough_score 决定 winner
    if score_a > score_b:
        winner = "A"
    elif score_b > score_a:
        winner = "B"
    else:
        # rough_score 相同，调用 LLM 比较

        # 生成 LLM 消息
        messages = build_pair_prompt(title_a, title_b, role="Aspiring Human Resources")

        # 调用 LLM 生成比较结果
        resp = run_llm(model, tokenizer, messages, max_tokens=5)
        resp = resp.strip().upper()

        # 判定胜者
        if resp == "A":
            winner = "A"
        elif resp == "B":
            winner = "B"
        else:
            # 异常或平局
            winner = "Tie"

    # 保存到列表
    pairwise_results.append({
        "A": title_a,
        "B": title_b,
        "winner": winner,
        "score_A": score_a,
        "score_B": score_b
    })

# 保存到 CSV 或 JSON
df_pairwise = pd.DataFrame(pairwise_results)
df_pairwise.to_csv("/content/drive/MyDrive/Apziva/df_train_pairwise.csv", index=False)
df_pairwise.to_json("/content/drive/MyDrive/Apziva/df_train_pairwise.json", orient="records", lines=True)

print(df_pairwise)
print(df_pairwise["winner"])
print(f"Saved {len(df_pairwise)} pairwise comparisons for LoRA training.")

Number of training titles: 50
                                                      A  \
0     Admissions Representative at Community medical...   
1     Admissions Representative at Community medical...   
2     Admissions Representative at Community medical...   
3     Admissions Representative at Community medical...   
4     Admissions Representative at Community medical...   
...                                                 ...   
1220  Senior Human Resources Business Partner at Hei...   
1221  Senior Human Resources Business Partner at Hei...   
1222  Aspiring Human Resources Manager, seeking inte...   
1223  Aspiring Human Resources Manager, seeking inte...   
1224  Aspiring Human Resources Professional | Passio...   

                                                      B winner  score_A  \
0                                               Student      B        1   
1     Native English Teacher at EPIK (English Progra...      A        1   
2                        Always set 

In [ ]:
train_dataset = []
for _, row in df_pairwise.iterrows():
    instruction = "Compare two job titles for HR role relevance and decide which candidate is more fit."
    input_text = f"A: {row['A']}\nB: {row['B']}"

    # winner 是 "A"/"B"，映射回对应 title
    if row['winner'] == "A":
        output_text = row['A']
    elif row['winner'] == "B":
        output_text = row['B']
    else:
        output_text = "Tie"  # 或者选择跳过此样本

    train_dataset.append({
        "instruction": instruction,
        "input": input_text,
        "output": output_text
    })

print(f"Number of training samples: {len(train_dataset)}")
train_dataset[:3]

Number of training samples: 1225


[{'instruction': 'Compare two job titles for HR role relevance and decide which candidate is more fit.',
  'input': 'A: Admissions Representative at Community medical center long beach\nB: Student',
  'output': 'Student'},
 {'instruction': 'Compare two job titles for HR role relevance and decide which candidate is more fit.',
  'input': 'A: Admissions Representative at Community medical center long beach\nB: Native English Teacher at EPIK (English Program in Korea)',
  'output': 'Admissions Representative at Community medical center long beach'},
 {'instruction': 'Compare two job titles for HR role relevance and decide which candidate is more fit.',
  'input': 'A: Admissions Representative at Community medical center long beach\nB: Always set them up for Success',
  'output': 'Admissions Representative at Community medical center long beach'}]

In [ ]:
import json

save_path = "/content/drive/MyDrive/Apziva/train_dataset.json"

with open(save_path, "w", encoding="utf-8") as f:
    json.dump(train_dataset, f, ensure_ascii=False, indent=2)

print(f"Training dataset saved to {save_path}")

import pandas as pd

df_train_dataset = pd.DataFrame(train_dataset)
csv_path = "/content/drive/MyDrive/Apziva/train_dataset.csv"
df_train_dataset.to_csv(csv_path, index=False)
print(f"Training dataset saved to {csv_path}")


Training dataset saved to /content/drive/MyDrive/Apziva/train_dataset.json
Training dataset saved to /content/drive/MyDrive/Apziva/train_dataset.csv


In [ ]:
# QLoRA (moved to initial model loading cell OoE9LH7Dc06I)